In [1]:
# Reexecutem després del reset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd

# ----------------------
# FUNCIONS AUXILIARS
# ----------------------
def generate_dataset(p, g):
    X = np.arange(1, p)
    log_table = {pow(g, i, p): i for i in range(1, p)}
    X_log = np.array([log_table[x] for x in X])

    s = np.random.randint(0, p - 2)
    r = (p - 3) // 2
    label_range = (s, s + r) if s + r < p - 1 else (s, (s + r) % (p - 1))

    def assign_label(log_val):
        if label_range[0] < label_range[1]:
            return 1 if label_range[0] <= log_val <= label_range[1] else -1
        else:
            return 1 if log_val >= label_range[0] or log_val <= label_range[1] else -1

    y = np.array([assign_label(val) for val in X_log])
    X_reshaped = X.reshape(-1, 1) / p
    return X_reshaped, y

# ----------------------
# BUCLE D'ITERACIONS
# ----------------------
p = 199
g = 3
n_iter = 20

best_acc_lr = 0
best_acc_svm = 0

for _ in range(n_iter):
    X, y = generate_dataset(p, g)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf_lr = LogisticRegression().fit(X_train, y_train)
    acc_lr = accuracy_score(y_test, clf_lr.predict(X_test))
    best_acc_lr = max(best_acc_lr, acc_lr)

    clf_svm = SVC(kernel='rbf').fit(X_train, y_train)
    acc_svm = accuracy_score(y_test, clf_svm.predict(X_test))
    best_acc_svm = max(best_acc_svm, acc_svm)

# ----------------------
# RESULTATS FINALS
# ----------------------
results_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'SVM (RBF kernel)'],
    'Best Accuracy': [best_acc_lr, best_acc_svm]
})

print(results_df)

                 Model  Best Accuracy
0  Logistic Regression       0.533333
1     SVM (RBF kernel)       0.550000


In [39]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Dataset i configuració
p = 199
g = 3
n_iter = 20
best_acc = 0

def custom_feature_map_3qubits(x, theta=np.pi/2):
    qc = QuantumCircuit(3)
    for i in range(3):
        qc.ry(theta, i)
        qc.rz(-2 * x[i], i)
        qc.rx(-2 * x[i], i)
    qc.cz(0, 1)
    qc.cz(1, 2)
    qc.cz(0, 2)
    return Statevector.from_instruction(qc)

def quantum_kernel_matrix(X1, X2, theta=np.pi/2):
    kernel = np.zeros((len(X1), len(X2)))
    for i in range(len(X1)):
        sv1 = custom_feature_map_3qubits(X1[i], theta)
        for j in range(len(X2)):
            sv2 = custom_feature_map_3qubits(X2[j], theta)
            kernel[i, j] = np.abs(sv1.data.conj().T @ sv2.data) ** 2
    return kernel

for _ in range(n_iter):
    # Genera dataset
    X = np.arange(1, p).reshape(-1, 1)
    log_table = {pow(g, i, p): i for i in range(1, p)}
    X_log = np.array([log_table[x[0]] for x in X])

    s = np.random.randint(0, p - 2)
    r = (p - 3) // 2
    label_range = (s, s + r) if s + r < p - 1 else (s, (s + r) % (p - 1))

    def assign_label(v):
        if label_range[0] < label_range[1]:
            return 1 if label_range[0] <= v <= label_range[1] else -1
        else:
            return 1 if v >= label_range[0] or v <= label_range[1] else -1

    y = np.array([assign_label(val) for val in X_log])

    # Escalat i duplicació
    X_scaled = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(X)
    X_dup = np.hstack([X_scaled]*3)

    # Train/test
    X_train, X_test, y_train, y_test = train_test_split(X_dup, y, test_size=0.3, random_state=42)

    # Kernel + SVM
    K_train = quantum_kernel_matrix(X_train, X_train)
    K_test = quantum_kernel_matrix(X_test, X_train)
    svm = SVC(kernel='precomputed')
    svm.fit(K_train, y_train)
    acc = accuracy_score(y_test, svm.predict(K_test))
    best_acc = max(best_acc, acc)

print(f"Millor accuracy amb Quantum Kernel personalitzat (3 qubits): {best_acc:.4f}")


Millor accuracy amb Quantum Kernel personalitzat (3 qubits): 0.6000
